In [1]:
%pip install Mastodon.py --upgrade

                                              0.0/65.9 kB ? eta -:--:--
     ---------------------------------------- 65.9/65.9 kB 1.7 MB/s eta 0:00:00
                                              0.0/409.3 kB ? eta -:--:--
     -----------------------                256.0/409.3 kB 7.9 MB/s eta 0:00:01
     -------------------------------------- 409.3/409.3 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: Mastodon.py
    Found existing installation: Mastodon.py 1.8.0
    Uninstalling Mastodon.py-1.8.0:
      Successfully uninstalled Mastodon.py-1.8.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## mastodon api access

- Create an app to access mastodon data 
- Collect followers given a user
- Collect profile information (if available, Twitter user) given a user


In [1]:
import glob
import parse
import pandas as pd


https://github.com/mastodon/mastodon/

https://mastodonpy.readthedocs.io/en/stable/02_return_values.html#user-account-dicts


In [5]:
from mastodon import Mastodon

mastodon_app = open("acmbrito_app_python_secret").readlines()
client_id, client_secret, token, api_base_url = mastodon_app
api1 = Mastodon(
    client_id=client_id, 
    client_secret=client_secret, 
    access_token=token,
    api_base_url=api_base_url
)

tutorial: https://gist.github.com/aparrish/661fca5ce7b4882a8c6823db12d42d26

In [6]:
def get_userid(user_name, user_server):
    try:
        user_profile = api1.account_lookup("{}@{}".format(user_name, user_server))
        return user_profile["id"]
    except Exception as error:
        print(error)
        return None


def get_userprofile(user_name, user_server):
    user_profile = api1.account_lookup("{}@{}".format(user_name, user_server))

    tt_user = ""
    for field in user_profile["fields"]:
        if field["name"] == "Twitter":
            tt_user = field["value"]

    bio = user_profile["note"]
    display_name = user_profile["display_name"]
    followers = user_profile["followers_count"]
    following = user_profile["following_count"]

    return (
        user_name,
        user_server,
        display_name,
        tt_user,
        followers,
        following,
        bio.replace("\t", " "),
    )


In [ ]:
def get_userfollowers(user_id):
    followers = api1.account_followers(user_id)
    followers_id = []
    count = 0
    while followers:
        count += 1
        if count > 3:
            break
        for f in followers:
            followers_id.append(f['id'])
        followers = api1.fetch_next(followers)

    return followers_id

user_name = "oseymour"
user_server = "sigmoid.social"
user_profile = api1.account_lookup("{}@{}".format(user_name, user_server))

In [ ]:
handle = ["mastodon", "account"]

files = glob.glob("csv/*.csv")
mastodon_users = dict()
for file in files:
    df = pd.read_csv(file, on_bad_lines="warn")
    for col in df.columns:
        if handle[0] in col.lower() or handle[1] in col.lower():
            scholars = df[col].values
            str_format = "csv\scholars_{}.csv"
            parsed = parse.parse(str_format, file)
            mastodon_users[parsed[0]] = scholars


In [ ]:
output = []
for field, scholars in mastodon_users.items():
    for handle in scholars:
        str_format = "{}@{}"
        if handle.startswith("@"):
            handle = handle[1:]
        parsed = parse.parse(str_format, handle)
        if parsed:
            user_name = parsed[0]
            user_server = parsed[1]

            user_id = get_userid(user_server, user_name)
            if user_id:
                followers = get_userfollowers(user_id)

                output.append((user_name, user_server, followers, field))


outfile = open("users_followers.csv", "w")
for line in output:
    outfile.write("{}\t{}\t{}\t{}\n".format(*line))
outfile.close()


In [8]:
seeds = ['@Nature@mstdn.social', '@PNASNews@fediscience.org', '@sciencemagazine@sciencemastodon.com']
seed_ids = []
for seed in seeds:
    parsed = parse.parse('@{}@{}', seed)
    user_id = get_userid(parsed[1], parsed[0])
    seed_ids.append(user_id)


def get_users(user_id, deep):
    if deep == 0:
        return set()
    else:
        followers = set(get_userfollowers(user_id))
        new_followers = set()
        for follower in followers:
            new_followers |= get_users(follower, deep - 1)
        return followers | new_followers
    
test = get_users(seed_ids[0], 2)

In [9]:
len(test)

1096